In [ ]:
import requests, json, pandas as pd

In [ ]:
def marketsMojoPeers(mm_stockId):
    """ 
    Returns the Quality, Valuation and Financial Trend with Short Term Technical Rating from Markets Mojo for all its peers
    
    Input--> Markets Mojo Stock ID  (Example mm_stockId - 592009 - HDFC BANK)
    
    Output --> Pandas Dataframe with Mojo Metrics
    
    
    """
    #import pandas as pd, requests, json
    url = "https://frapi.marketsmojo.com/stocks/getPeersDetails"

    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0',
      'Accept': 'application/json, text/plain, */*',
      'Accept-Language': 'en-US,en;q=0.5',
      'Content-Type': 'text/plain',
      'Origin': 'https://www.marketsmojo.com',
      'DNT': '1',
      'Connection': 'keep-alive',
      'Referer': 'https://www.marketsmojo.com/',
      'TE': 'Trailers',
      'Pragma': 'no-cache',
      'Cache-Control': 'no-cache'
    }
    try:
        payload = '{\"stock_id\":\"'+ mm_stockId + '\",\"popup\":1}'
        
        response = requests.request("POST", url, headers=headers, data = payload)
        
        stock_list = {}
        
        for each_stock in response.json()['data']['peer']['list']: 
                stock = {}
                stock['Name'] = each_stock['details']['sname']
                stock['Industry'] = each_stock['details']['ind_name']
                stock['quality'] = each_stock['dot']['q_txt']
                stock['valuation'] = each_stock['dot']['v_txt']
                stock['financial'] = each_stock['dot']['f_txt']
                stock['technical'] = each_stock['dot']['f_txt']
                stock['Mojo'] = each_stock['dot']['scoreText']
                stock_list[each_stock['details']['sid']] = stock
                
        mojo_temp = pd.DataFrame.from_dict(stock_list).transpose()
        
        return mojo_temp
    
    except:
        print('Unable to find the Stock with Markets Mojo Stock ID %s' %(mm_stockId)  )


In [ ]:
mojo = pd.read_csv("MojoWithCodes.csv")
mmid = mojo.groupby('Industry').first()['mm_id']
mojo_temp = marketsMojoPeers('643071')

for industry, stock_id in mmid.iteritems():
    print(f'Getting Peer Group for {industry}')
    mojo_temp = mojo_temp.append(marketsMojoPeers(str(stock_id)))
mojo = pd.read_csv("MojoWithCodes.csv", index_col='mm_id')    
mojo_temp = mojo[['BSEID', 'NSEID']].join(mojo_temp)

In [ ]:
mojo_temp.drop_duplicates(inplace=True)
mojo_temp[(mojo_temp['Mojo'] == 'Strong Buy')].sort_values(by=['Industry','Name'])